In [1]:
import os
import pandas as pd
import numpy as np
from os import walk
import collections

In [2]:
# Path of all files in folder and subfolder - CONTINUE Folder
List_of_files_continue = []
folder_continue = "C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/dailyContinue/"
folder_names_continue = [os.path.join(folder_continue, name) for name in os.listdir(folder_continue) if os.path.isdir(os.path.join(folder_continue, name))]
folder_names_continue.append(folder_continue)
for fold in folder_names_continue:
    filenames = next(walk(fold), (None, None, []))[2]  # [] if no file
    for file in filenames:
        List_of_files_continue.append(str(fold+"/"+file).replace('//','/'))
List_of_files_continue = [val for val in List_of_files_continue if not val.endswith(".ipynb")] # remove all files end in .ipynb
List_of_files_continue = [val for val in List_of_files_continue if not val.endswith(".pkl")] # remove all files end in .pkl
List_of_files_continue = [val for val in List_of_files_continue if not val.endswith("expiry_df_other_symbols_Continue.csv")]
List_of_files_continue = [val for val in List_of_files_continue if not val.endswith("expiry_df_Continue.csv")]

len(List_of_files_continue)

621

In [3]:
# Path of all files in folder and subfolder - NEAREST Folder
List_of_files_nearest = []
folder_nearest = "C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/"
folder_names_nearest = [os.path.join(folder_nearest, name) for name in os.listdir(folder_nearest) if os.path.isdir(os.path.join(folder_nearest, name))]
folder_names_nearest.append(folder_nearest)
folder_names_nearest.remove("C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/dailyContinue")
for fold in folder_names_nearest:
    filenames = next(walk(fold), (None, None, []))[2]  # [] if no file
    for file in filenames:
        List_of_files_nearest.append(str(fold+"/"+file).replace('//','/'))
List_of_files_nearest = [val for val in List_of_files_nearest if not val.endswith(".ipynb")] # remove all files end in .ipynb
List_of_files_nearest = [val for val in List_of_files_nearest if not val.endswith(".pkl")] # remove all files end in .pkl
List_of_files_nearest = [val for val in List_of_files_nearest if not val.endswith("combined_dataframe.csv")]
List_of_files_nearest = [val for val in List_of_files_nearest if not val.endswith("expiry_df.csv")]
List_of_files_nearest = [val for val in List_of_files_nearest if not val.endswith("expiry_df_other_symbols.csv")]

len(List_of_files_nearest)

655

In [4]:
# Matching Nearest & Continue FILES - just remove "_Continue" at the end but before ".csv"
# For each file in Nearest, change the string of the path to make it similar to one in Continue and the check if its exits in Continue

file_paths_dict = {}
for file_nearest in List_of_files_nearest:
    continue_string = file_nearest[file_nearest.rindex('/')+1:]
    continue_string = str(continue_string.replace(".csv","")+"_Continue.csv")
    idx_continue = [i for i, s in enumerate(List_of_files_continue) if continue_string in s]
    file_paths_dict[file_nearest] = List_of_files_continue[idx_continue[0]]

len(file_paths_dict.keys())    
        

655

In [5]:
# Get Expiry dataframe from csv
expiry_nearest = pd.read_csv("C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/expiry_df.csv", index_col=0)
expiry_nearest.head(2)   

,symbol,exchange,contract,firstNoticeDate,expirationDate,lastTradingDay,Unnamed: 7
MGM22,MGM22,GBLX,_Jun 22,6/13/2022,6/13/2022,Second business day preceding third Wednesday ...,NaN
MGH22,MGH22,GBLX,_Mar 22,3/14/2022,3/14/2022,Second business day preceding third Wednesday ...,NaN


In [6]:
# Collect all contracts for all commodities in a dictionary and a list of dates
all_dates = []
cmdty_ctrct_dict = {}
cmdty_no_data = []
temp_list = []
cmdty_no_expiry = []
cmdty_ctrct_expiry_dict = collections.defaultdict(dict) # If the amount of nesting you need is fixed, use collections.defaultdict.

for file in file_paths_dict.keys():
    cmdty_name = file[file.rindex('/')+1:].replace('_daily_price.csv','')
    
    try:
        list_nearest_ctrct = pd.read_csv(file)['symbol'].unique().tolist()
        list_nearest_dates = pd.read_csv(file)['tradingDay'].unique().tolist()

    except:
        cmdty_no_data.append(cmdty_name)
        temp_list.append(file_paths_dict[file])

    try:
        list_continue_ctrct = pd.read_csv(file_paths_dict[file],index_col=0)['symbol'].unique().tolist()
        list_continue_dates = pd.read_csv(file_paths_dict[file],index_col=0)['tradingDay'].unique().tolist()

    except:
        cmdty_no_data.append(cmdty_name)
        temp_list.append(file_paths_dict[file])

    cmdty_ctrct_dict[cmdty_name] = set(list_nearest_ctrct + list_continue_ctrct)

    all_dates = all_dates + list_nearest_dates + list_continue_dates

    for ctrcts in cmdty_ctrct_dict[cmdty_name]:
        try:
            cmdty_ctrct_expiry_dict[cmdty_name][ctrcts] = expiry_nearest.loc[ctrcts,'expirationDate'], expiry_nearest.loc[ctrcts,'exchange'], expiry_nearest.loc[ctrcts,'contract'].replace('_','')
        except:
            cmdty_no_expiry.append(ctrcts)

        # To do dict of dicts
	
temp_list =  set(temp_list)
all_dates = set(all_dates)
cmdty_no_data = set(cmdty_no_data)
all_dates = pd.Series(pd.DatetimeIndex(all_dates))
all_dates =  all_dates.sort_values(ascending=False).dt.strftime('%Y-%m-%d').tolist()  


In [7]:

len(cmdty_ctrct_dict.keys()), len(list(cmdty_ctrct_expiry_dict.keys())), len(cmdty_no_expiry)

(618, 561, 6141)

In [8]:
cmdty_ctrct_expiry_dict['AUD_USD Micro']['MGZ20'][2]

'Dec 20'

In [9]:
len(cmdty_no_data), len(file_paths_dict.keys()), len(cmdty_ctrct_dict.keys())

(110, 655, 618)

In [10]:
LS_path = 'C:/source/2x4_data/app/data/Barchart/DailyDataDumpCME&NYMEX/Lumber_daily_price.csv'
file = LS_path
combined = pd.concat([pd.read_csv(file,index_col=0), pd.read_csv(file_paths_dict[file],index_col=0)])
combined.drop_duplicates(inplace=True)
len(combined)


157047

In [11]:
one_commodity_expiry = pd.DataFrame(cmdty_ctrct_expiry_dict['Lumber']).T
one_commodity_expiry.columns = ['expiry','exchange','contract']
one_commodity_expiry.expiry = pd.to_datetime(one_commodity_expiry.expiry)
one_commodity_expiry.sort_values(by=['expiry'], ascending= False, inplace=True)
len(one_commodity_expiry)


321

In [12]:
combined_pivot = pd.pivot_table(combined, values='close', index='tradingDay', columns='symbol').sort_index(ascending=False)
combined_pivot.reindex(one_commodity_expiry.index, axis=1)
len(combined_pivot.columns)


321

In [13]:
# same as commodity_pivot but added an extra row at the top for the expiry date
new_combined_pivot = pd.concat([pd.DataFrame(index = one_commodity_expiry.index, data=list(one_commodity_expiry['expiry']),columns=['Expiry']).T,combined_pivot], axis=0)
len(new_combined_pivot)

13261

In [14]:
# new_combined_pivot.to_csv('C:/source/2x4_data/app/data/Barchart/Lumber_Final_DataFrame.csv')

In [15]:
# Takes a lot of Time - ~ 2minutes for 1 commodity because OF nsmallest(NearestMonth), sort_values is faster ~40 seconds
# Get Front month next to expiry
NearestMonth = 1  # starts from month 1 
NearestContract_df = pd.DataFrame()
for dt in new_combined_pivot.index[1:]:

    comp = pd.to_datetime(new_combined_pivot.loc['Expiry'])[dt <= pd.to_datetime(new_combined_pivot.loc['Expiry'])] - pd.to_datetime(dt)
    # active = comp.nsmallest(NearestMonth).index[NearestMonth-1]
    active = comp.sort_values().head(NearestMonth).index[NearestMonth-1]
    temp3 = pd.DataFrame(data=[[dt,active]], columns=['dt','active'])
    NearestContract_df = pd.concat([NearestContract_df,temp3])

In [16]:
NearestContract_df.iloc[39][1], NearestContract_df.iloc[39][0]

('LSH22', '2022-03-15')

In [17]:
new_combined_pivot.loc[NearestContract_df.iloc[39][0],NearestContract_df.iloc[39][1]]

1420.0

In [18]:
some_dt = '2022-03-15'
NearestContract_df[NearestContract_df['dt'] == some_dt]

,dt,active
0,2022-03-15,LSH22


In [19]:
NearestContract_df['close'] = np.nan
NearestContract_df.reset_index(inplace = True)
NearestContract_df.drop('index', axis=1,inplace = True)
for i in range(len(NearestContract_df)):
    NearestContract_df.loc[i, 'close'] = new_combined_pivot.loc[NearestContract_df.iloc[i][0],NearestContract_df.iloc[i][1]]

NearestContract_df

,dt,active,close
0,2022-05-10,LSK22,1022.0
1,2022-05-09,LSK22,1009.0
2,2022-05-06,LSK22,1000.1
3,2022-05-05,LSK22,1020.0
4,2022-05-04,LSK22,1010.0
...,...,...,...
13255,1969-10-07,LSH70,79.0
13256,1969-10-06,LSH70,80.0
13257,1969-10-03,LSH70,80.0
13258,1969-10-02,LSH70,80.5


In [20]:
backAdjustDays = 2       # 0 means expired on the day of expiry
NearestContractShifted_df = NearestContract_df.copy(deep=True)
shifted_contracts = pd.Series(NearestContractShifted_df.active[0:backAdjustDays+1].values)
NearestContractShifted_df.active = NearestContractShifted_df.active.shift(periods=(backAdjustDays+1))
NearestContractShifted_df.active.iloc[0:backAdjustDays+1] = shifted_contracts
NearestContractShifted_df.active.reset_index(drop=True, inplace=True)

for i in range(len(NearestContractShifted_df)):
    NearestContractShifted_df.loc[i, 'close'] = new_combined_pivot.loc[NearestContractShifted_df.iloc[i][0],NearestContractShifted_df.iloc[i][1]]

NearestContractShifted_df

C:\Users\Fanjum_cwp\AppData\Local\Temp\2\ipykernel_17640\1244273054.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NearestContractShifted_df.active.iloc[0:backAdjustDays+1] = shifted_contracts


,dt,active,close
0,2022-05-10,LSK22,1022.0
1,2022-05-09,LSK22,1009.0
2,2022-05-06,LSK22,1000.1
3,2022-05-05,LSK22,1020.0
4,2022-05-04,LSK22,1010.0
...,...,...,...
13255,1969-10-07,LSH70,79.0
13256,1969-10-06,LSH70,80.0
13257,1969-10-03,LSH70,80.0
13258,1969-10-02,LSH70,80.5


In [21]:
NearestContractShifted_df[NearestContractShifted_df.close.isna()][:5]

,dt,active,close
6363,1997-01-31,LSH97,NaN
10106,1982-04-07,LSZ82,NaN
10107,1982-04-06,LSZ82,NaN
10108,1982-04-05,LSZ82,NaN
10109,1982-04-02,LSZ82,NaN


In [22]:
NearestContract_df[(NearestContract_df.dt == '1982-04-02') & (NearestContract_df.active == 'LSZ82')]

,dt,active,close
10109,1982-04-02,LSZ82,NaN


In [23]:
NearestContractShifted_df[38:41]

,dt,active,close
38,2022-03-16,LSK22,1150.0
39,2022-03-15,LSK22,1160.4
40,2022-03-14,LSK22,1103.4


In [24]:
NearestContract_df[38:41]

,dt,active,close
38,2022-03-16,LSK22,1150.0
39,2022-03-15,LSH22,1420.0
40,2022-03-14,LSH22,1410.0


In [25]:
len(NearestContract_df), len(NearestContractShifted_df)

(13260, 13260)

In [26]:

NearestContract_df.index = NearestContract_df.dt
NearestContract_df.drop('dt', axis=1,inplace = True)

NearestContractShifted_df.index = NearestContractShifted_df.dt
NearestContractShifted_df.drop('dt', axis=1,inplace = True)

ComparisonNearestContractdf=NearestContract_df.compare(NearestContractShifted_df)
ComparisonNearestContractdf

active           close        
             self  other     self   other
dt                                       
2022-03-15  LSH22  LSK22  1420.00  1160.4
2022-03-14  LSH22  LSK22  1410.00  1103.4
2022-03-11  LSH22  LSK22  1410.00  1159.2
2022-01-14  LSF22  LSH22  1237.10  1308.7
2022-01-13  LSF22  LSH22  1229.10  1329.0
...           ...    ...      ...     ...
1970-05-14  LSK70  LSN70    73.50    77.5
1970-05-13  LSK70  LSN70    73.25    78.5
1970-03-13  LSH70  LSK70    70.50    77.5
1970-03-12  LSH70  LSK70    75.50    79.0
1970-03-11  LSH70  LSK70    75.50    79.5

[942 rows x 4 columns]

In [27]:

ComparisonNearestContractdf['Diff'] = ComparisonNearestContractdf.loc[:,('close','other')].sub(ComparisonNearestContractdf.loc[:,('close','self')], axis = 0)
ComparisonNearestContractdf['Diff']


dt
2022-03-15   -259.60
2022-03-14   -306.60
2022-03-11   -250.80
2022-01-14     71.60
2022-01-13     99.90
               ...  
1970-05-14      4.00
1970-05-13      5.25
1970-03-13      7.00
1970-03-12      3.50
1970-03-11      4.00
Name: Diff, Length: 942, dtype: float64

In [28]:
ComparisonNearestContractdf

active           close            Diff
             self  other     self   other        
dt                                               
2022-03-15  LSH22  LSK22  1420.00  1160.4 -259.60
2022-03-14  LSH22  LSK22  1410.00  1103.4 -306.60
2022-03-11  LSH22  LSK22  1410.00  1159.2 -250.80
2022-01-14  LSF22  LSH22  1237.10  1308.7   71.60
2022-01-13  LSF22  LSH22  1229.10  1329.0   99.90
...           ...    ...      ...     ...     ...
1970-05-14  LSK70  LSN70    73.50    77.5    4.00
1970-05-13  LSK70  LSN70    73.25    78.5    5.25
1970-03-13  LSH70  LSK70    70.50    77.5    7.00
1970-03-12  LSH70  LSK70    75.50    79.0    3.50
1970-03-11  LSH70  LSK70    75.50    79.5    4.00

[942 rows x 5 columns]

In [29]:
diff_df = pd.DataFrame(index = NearestContract_df.index, data = ComparisonNearestContractdf['Diff'])
# diff_df.reset_levels(0, inplace=True)
diff_df.fillna(method='ffill',inplace = True)
diff_df.fillna(0,inplace = True)
diff_df

,Diff
dt,
2022-05-10,0.0
2022-05-09,0.0
2022-05-06,0.0
2022-05-05,0.0
2022-05-04,0.0
...,...
1969-10-07,4.0
1969-10-06,4.0
1969-10-03,4.0


In [30]:
diff_df[38:]

,Diff
dt,
2022-03-16,0.0
2022-03-15,-259.6
2022-03-14,-306.6
2022-03-11,-250.8
2022-03-10,-250.8
...,...
1969-10-07,4.0
1969-10-06,4.0
1969-10-03,4.0


In [31]:
ComparisonNearestContractdf.index

Index(['2022-03-15', '2022-03-14', '2022-03-11', '2022-01-14', '2022-01-13',
       '2022-01-12', '2021-11-15', '2021-11-12', '2021-11-11', '2021-09-15',
       ...
       '1970-09-11', '1970-07-15', '1970-07-14', '1970-07-13', '1970-05-15',
       '1970-05-14', '1970-05-13', '1970-03-13', '1970-03-12', '1970-03-11'],
      dtype='object', name='dt', length=942)

In [32]:
cum_diff_df = pd.DataFrame(index = ComparisonNearestContractdf.index, data = ComparisonNearestContractdf['Diff'])
cum_diff_df

,Diff
dt,
2022-03-15,-259.60
2022-03-14,-306.60
2022-03-11,-250.80
2022-01-14,71.60
2022-01-13,99.90
...,...
1970-05-14,4.00
1970-05-13,5.25
1970-03-13,7.00


In [33]:
cum_diff_df.reset_index(inplace=True)
cum_diff_df


,dt,Diff
0,2022-03-15,-259.60
1,2022-03-14,-306.60
2,2022-03-11,-250.80
3,2022-01-14,71.60
4,2022-01-13,99.90
...,...,...
937,1970-05-14,4.00
938,1970-05-13,5.25
939,1970-03-13,7.00
940,1970-03-12,3.50


In [34]:
cum_diff_df['cumsum'] = cum_diff_df['Diff'].groupby((cum_diff_df.index % (backAdjustDays+1))== backAdjustDays).cumsum()
cum_diff_df

,dt,Diff,cumsum
0,2022-03-15,-259.60,-259.60
1,2022-03-14,-306.60,-566.20
2,2022-03-11,-250.80,-250.80
3,2022-01-14,71.60,-494.60
4,2022-01-13,99.90,-394.70
...,...,...,...
937,1970-05-14,4.00,21.28
938,1970-05-13,5.25,14.98
939,1970-03-13,7.00,28.28
940,1970-03-12,3.50,31.78


In [35]:
# cum_diff_df.to_csv('C:/source/2x4_data/app/data/Barchart/cum_diff.csv')

In [36]:
# cum_diff_df['Diff'].groupby((cum_diff_df.index % (backAdjustDays+1))== 1).cumsum()

In [37]:
# (cum_diff_df.index % (backAdjustDays+1) != backAdjustDays)

In [38]:
cum_diff_df['Diff'].index

RangeIndex(start=0, stop=942, step=1)

In [39]:
# cum_diff_df['Diff'].apply(lambda x: x.cumsum() if ((cum_diff_df.index % (backAdjustDays+1)) - backAdjustDays).all() else 0)


In [40]:
# cum_diff_df

In [41]:
pivot_backAdjustDays_idx = [x for x in cum_diff_df.index if (x % (backAdjustDays+1)) == backAdjustDays]

In [42]:
cum_diff_df['flag'] = [1 if x in pivot_backAdjustDays_idx else 0 for x in cum_diff_df.index]
cum_diff_df

,dt,Diff,cumsum,flag
0,2022-03-15,-259.60,-259.60,0
1,2022-03-14,-306.60,-566.20,0
2,2022-03-11,-250.80,-250.80,1
3,2022-01-14,71.60,-494.60,0
4,2022-01-13,99.90,-394.70,0
...,...,...,...,...
937,1970-05-14,4.00,21.28,0
938,1970-05-13,5.25,14.98,1
939,1970-03-13,7.00,28.28,0
940,1970-03-12,3.50,31.78,0


In [43]:
cum_diff_df['cumsum2'] =  (cum_diff_df['Diff']*cum_diff_df['flag']).cumsum()

In [44]:
cum_diff_df['flag'] = 1 - cum_diff_df['flag']
cum_diff_df['True_cumsum'] = (cum_diff_df['Diff']*cum_diff_df['flag']) + cum_diff_df['cumsum2']


In [45]:
cum_diff_df.index  = cum_diff_df.dt
cum_diff_df.drop('dt', axis=1,inplace = True)
cum_diff_df

,Diff,cumsum,flag,cumsum2,True_cumsum
dt,,,,,
2022-03-15,-259.60,-259.60,1,-0.00,-259.60
2022-03-14,-306.60,-566.20,1,-0.00,-306.60
2022-03-11,-250.80,-250.80,0,-250.80,-250.80
2022-01-14,71.60,-494.60,1,-250.80,-179.20
2022-01-13,99.90,-394.70,1,-250.80,-150.90
...,...,...,...,...,...
1970-05-14,4.00,21.28,1,9.73,13.73
1970-05-13,5.25,14.98,0,14.98,14.98
1970-03-13,7.00,28.28,1,14.98,21.98


In [46]:
# NearestContract_df
ComparisonNearestContractdf[250:]

active          close          Diff
             self  other    self  other       
dt                                            
2008-05-14  LSK08  LSN08  225.90  242.5  16.60
2008-05-13  LSK08  LSN08  226.60  244.9  18.30
2008-03-14  LSH08  LSK08  188.00  230.7  42.70
2008-03-13  LSH08  LSK08  185.90  226.9  41.00
2008-03-12  LSH08  LSK08  190.80  228.7  37.90
...           ...    ...     ...    ...    ...
1970-05-14  LSK70  LSN70   73.50   77.5   4.00
1970-05-13  LSK70  LSN70   73.25   78.5   5.25
1970-03-13  LSH70  LSK70   70.50   77.5   7.00
1970-03-12  LSH70  LSK70   75.50   79.0   3.50
1970-03-11  LSH70  LSK70   75.50   79.5   4.00

[692 rows x 5 columns]

In [47]:
cum_diff_df['True_cumsum']

dt
2022-03-15   -259.60
2022-03-14   -306.60
2022-03-11   -250.80
2022-01-14   -179.20
2022-01-13   -150.90
               ...  
1970-05-14     13.73
1970-05-13     14.98
1970-03-13     21.98
1970-03-12     18.48
1970-03-11     18.98
Name: True_cumsum, Length: 942, dtype: float64

In [48]:
final_df = pd.DataFrame(index = NearestContract_df.index, data = cum_diff_df['True_cumsum'] )
final_df.fillna(method='ffill',inplace = True)
final_df.fillna(0,inplace = True)
final_df['close'] = NearestContract_df['close'].values
final_df['Adjusted'] = final_df['close'] + final_df['True_cumsum']
final_df


,True_cumsum,close,Adjusted
dt,,,
2022-05-10,0.00,1022.0,1022.00
2022-05-09,0.00,1009.0,1009.00
2022-05-06,0.00,1000.1,1000.10
2022-05-05,0.00,1020.0,1020.00
2022-05-04,0.00,1010.0,1010.00
...,...,...,...
1969-10-07,18.98,79.0,97.98
1969-10-06,18.98,80.0,98.98
1969-10-03,18.98,80.0,98.98


In [49]:
final_df[78:]

,True_cumsum,close,Adjusted
dt,,,
2022-01-18,-250.80,1278.7,1027.90
2022-01-14,-179.20,1237.1,1057.90
2022-01-13,-150.90,1229.1,1078.20
2022-01-12,-178.80,1212.0,1033.20
2022-01-11,-178.80,1204.9,1026.10
...,...,...,...
1969-10-07,18.98,79.0,97.98
1969-10-06,18.98,80.0,98.98
1969-10-03,18.98,80.0,98.98


In [50]:
final_df.to_csv('C:/source/2x4_data/app/data/Barchart/final_df_2dayb4.csv')

In [51]:
final_df.loc['2001-04-30']

True_cumsum   -248.2
close          288.4
Adjusted        40.2
Name: 2001-04-30, dtype: float64

In [52]:
# final_df = cum_diff_df['True_cumsum'].ffill()
final_df

,True_cumsum,close,Adjusted
dt,,,
2022-05-10,0.00,1022.0,1022.00
2022-05-09,0.00,1009.0,1009.00
2022-05-06,0.00,1000.1,1000.10
2022-05-05,0.00,1020.0,1020.00
2022-05-04,0.00,1010.0,1010.00
...,...,...,...
1969-10-07,18.98,79.0,97.98
1969-10-06,18.98,80.0,98.98
1969-10-03,18.98,80.0,98.98


In [53]:
cum_diff_df[250:]

,Diff,cumsum,flag,cumsum2,True_cumsum
dt,,,,,
2008-05-14,16.60,-1183.30,1,-439.00,-422.40
2008-05-13,18.30,-420.70,0,-420.70,-420.70
2008-03-14,42.70,-1140.60,1,-420.70,-378.00
2008-03-13,41.00,-1099.60,1,-420.70,-379.70
2008-03-12,37.90,-382.80,0,-382.80,-382.80
...,...,...,...,...,...
1970-05-14,4.00,21.28,1,9.73,13.73
1970-05-13,5.25,14.98,0,14.98,14.98
1970-03-13,7.00,28.28,1,14.98,21.98


In [54]:
# pd.MultiIndex.from_frame(ComparisonNearestContractdf)

ComparisonNearestContractdf.columns.get_level_values(0), ComparisonNearestContractdf.columns.get_level_values(1)

(Index(['active', 'active', 'close', 'close', 'Diff'], dtype='object'),
 Index(['self', 'other', 'self', 'other', ''], dtype='object'))

In [55]:
ComparisonNearestContractdf.iloc[:, ComparisonNearestContractdf.columns.get_level_values(1)=='self']

,active,close
,self,self
dt,,
2022-03-15,LSH22,1420.00
2022-03-14,LSH22,1410.00
2022-03-11,LSH22,1410.00
2022-01-14,LSF22,1237.10
2022-01-13,LSF22,1229.10
...,...,...
1970-05-14,LSK70,73.50
1970-05-13,LSK70,73.25


In [56]:
# Check if any of the expiries are less than 2 std dev away from the mean of the date difference between 2 consecutive expiry months

def zscore(s):
    return (s - np.mean(s)) / np.std(s)

# Check if difference in dates is more or less similar - left starting 2 dates - becasue one will be NaN and other is current day so wil be less
expiry_month_diff = pd.to_datetime(new_combined_pivot.loc['Expiry']).diff().dt.days[2:].values
pd.Series(expiry_month_diff)[zscore(pd.Series(expiry_month_diff)).abs().gt(2)]

245   -14.0
246   -46.0
dtype: float64

In [57]:
columns = ['Cmdty','M1','M2','M3','M4','M5','M6','M7','M8','M9','M10','M11','M12']
one_cmdty_df = pd.DataFrame(columns=columns, index=all_dates)
cmdty_calender_mnths = pd.DataFrame(columns=columns)

In [58]:
cmdty_calender_mnths

,Cmdty,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12


In [59]:
# cmdty_ctrct_dict['AUD_USD Micro']
cmdty_ctrct_expiry_dict['AUD_USD Micro']['MGM21']

('6/14/2021', 'GBLX', 'Jun 21')

In [60]:
cmdty_date_df = pd.DataFrame(columns=['Dates'])
cmdty_date_df.loc['2','Dates'] = 1
# cmdty_date_df

In [61]:
import datetime
def date_less_than(date, ctrct_expiry):
    return date <= ctrct_expiry 

# def date_greater_than(end, current):
#     return current <= end

stick together continue & nearest contracts